In [1]:
import numpy as np
import tensorflow as tf

import time

In [2]:
batch_size = 32
n_epochs = 50
T = 24
D = 122
D1 = 100
n_users = 226791
vj_size = 20
Wu_size = 100
reg = 0.01
p = 0.2

In [3]:
import json
with open('./labels.json') as json_data:
    labels = json.load(json_data)


In [4]:
with open('./labeltest.json') as json_data:
    labels_test = json.load(json_data)

In [5]:
mask = np.load('./mask.npy')
mask.shape
mask = mask.T
mask.shape

(226791, 792)

In [6]:
maskTest = np.load('./mask_test.npy')
maskTest = maskTest.T
maskTest.shape

(226791, 199)

In [7]:
X1 = np.load('./finalscore_y.npy')
X1.shape

(226791, 50)

In [8]:
import re
X = np.zeros((792, 24, 122))
i = 0
label = []
import os
for e in os.listdir('./_FinalInputVectors/'):
    label.append(e)
    x = np.load('./_FinalInputVectors/'+e)
    X[i] = x[:24]
    i = i+1
print (X.shape)

(792, 24, 122)


In [9]:
import re
X_test = np.zeros((199, 24, 122))
i = 0
label_test = []
import os
for e in os.listdir('./FinalInputTestVectors/'):
    label_test.append(e)
    x = np.load('./FinalInputTestVectors/'+e)
    X_test[i] = x[:24]
    i = i+1
print (X_test.shape)

(199, 24, 122)


In [10]:
X= X.swapaxes(0,1)
X.shape

(24, 792, 122)

In [11]:
X_test= X_test.swapaxes(0,1)
X_test.shape

(24, 199, 122)

In [12]:
eventId = []
for l in label:
    regex =re.compile(r'\d+')
    eventId.append(regex.findall(l))
eventId = [item for sublist in eventId for item in sublist]

In [13]:
testeventId = []
for l in label_test:
    regex =re.compile(r'\d+')
    testeventId.append(regex.findall(l))
testeventId = [item for sublist in testeventId for item in sublist]

In [14]:
testeventId

['277',
 '870',
 '451',
 '645',
 '804',
 '116',
 '965',
 '249',
 '91',
 '382',
 '635',
 '543',
 '42',
 '583',
 '337',
 '274',
 '908',
 '431',
 '607',
 '158',
 '166',
 '937',
 '987',
 '743',
 '283',
 '961',
 '167',
 '782',
 '515',
 '125',
 '766',
 '425',
 '814',
 '237',
 '604',
 '403',
 '279',
 '364',
 '386',
 '673',
 '670',
 '514',
 '316',
 '401',
 '798',
 '224',
 '519',
 '655',
 '760',
 '983',
 '513',
 '920',
 '223',
 '39',
 '66',
 '254',
 '979',
 '982',
 '432',
 '688',
 '902',
 '319',
 '984',
 '853',
 '379',
 '426',
 '192',
 '689',
 '770',
 '817',
 '376',
 '90',
 '809',
 '208',
 '275',
 '497',
 '370',
 '971',
 '634',
 '900',
 '641',
 '653',
 '875',
 '628',
 '615',
 '507',
 '862',
 '617',
 '856',
 '988',
 '94',
 '659',
 '973',
 '542',
 '917',
 '87',
 '81',
 '399',
 '611',
 '833',
 '713',
 '180',
 '606',
 '704',
 '329',
 '281',
 '60',
 '32',
 '381',
 '651',
 '919',
 '779',
 '452',
 '293',
 '892',
 '647',
 '220',
 '327',
 '149',
 '429',
 '428',
 '512',
 '99',
 '530',
 '342',
 '576',
 '2

In [15]:
label = []
for e in eventId:
    label.append(int(labels[e]))

In [16]:
label_test = []
for e in testeventId:
    label_test.append(int(labels_test[e]))

In [17]:
len(label_test) # this is my ground truth label

199

In [18]:
label = np.array(label)
label.shape

(792,)

In [19]:
label_test = np.array(label)
label_test.shape

(792,)

In [20]:
#X = np.random.randn(T, batch_size, D)
#Y = np.random.randint(1, size = 992)
#X1 = np.random.rand(300, 20) # user y matrix
#mask = np.random.randint(2, size=(300, batch_size))

In [21]:
class Capture_Score(object):
    
    def __init__(self):
        self.batch_size = batch_size
        self.gstep = tf.Variable(0, dtype=tf.int32, trainable=False, name = 'global_step')
        self.lr = 0.001
        self.lstm_size = 50
        self.training = False
    def get_data(self):
        with tf.name_scope('data'):
            self.x = tf.placeholder(dtype=tf.float32, shape=[T, self.batch_size, D])
            self.y = tf.placeholder(dtype= tf.int32, shape=[self.batch_size])
        #with tf.name_scope('data_score'):
            self.x1 = tf.placeholder(dtype=tf.float32, shape=[n_users, 50])
            self.mask = tf.placeholder(dtype=tf.float32, shape=[n_users, self.batch_size])
    
    def inference(self):
        with tf.variable_scope('Capture'):
            Wa = tf.get_variable('Wa', shape=[D, D1], initializer=tf.random_normal_initializer)
            ba = tf.get_variable('ba', shape= [D1], initializer=tf.random_normal_initializer)  
            
           # dropout_Wa = tf.nn.dropout(Wa, keep_prob=p) * p
            dropout_Wa = tf.layers.dropout(inputs=Wa, rate=p, training=self.training)
            x_reshape = tf.matmul(tf.reshape(self.x, [-1, D]), dropout_Wa) + ba
            x1 = tf.reshape(x_reshape, [-1, self.batch_size, D1])
            D1_new = x1.shape[-1]
            weights = tf.get_variable('weights', shape = [D1_new, self.lstm_size],
                                  initializer=tf.random_normal_initializer)
            bias = tf.get_variable('bias', shape = [self.lstm_size], initializer=tf.random_normal_initializer)
        
            Wr = tf.get_variable('Wr', shape=[self.lstm_size, vj_size],initializer=tf.random_normal_initializer)
            br = tf.get_variable('br', shape= [vj_size], initializer=tf.random_normal_initializer)   
            
            #dropout_Wr = tf.nn.dropout(Wr, keep_prob=p) * p  #changed here for dropout
            dropout_Wr = tf.layers.dropout(inputs=Wr, rate=p, training=self.training)
            _x1 = tf.transpose(x1, [1,0,2])
            _x1 = tf.reshape(_x1, [-1, D1])

            _x1 = tf.tanh(tf.matmul(_x1, weights) + bias)
            _x1 = tf.split(_x1, T, 0)

            lstm_cell = tf.contrib.rnn.BasicLSTMCell(self.lstm_size)

            _, states = tf.contrib.rnn.static_rnn(lstm_cell, _x1, dtype=tf.float32)

            lstm_last_state = states[-1]

            vj = tf.matmul(lstm_last_state, dropout_Wr) + br
            capture_vector_shape = vj.shape[-1]
        
        #for score
        with tf.variable_scope('Score'):
            Wu = tf.get_variable(name='Wu', shape=[50, Wu_size], initializer=tf.random_normal_initializer,
                                regularizer = tf.contrib.layers.l2_regularizer(scale=reg))
            bu = tf.get_variable('bu', shape= [Wu_size], initializer=tf.random_normal_initializer)

            Ws = tf.get_variable('Ws', shape=[1,Wu_size], initializer = tf.random_normal_initializer)
            bs = tf.get_variable('bs', shape=[1], initializer=tf.random_normal_initializer)
        
            y1 = tf.tanh(tf.matmul(self.x1, Wu) + bu)

            s = tf.sigmoid(tf.reduce_sum(tf.multiply(y1,Ws), 1) + bs)

            s = tf.multiply(s, tf.transpose(self.mask))

            s= tf.reshape(s, shape = [self.batch_size, n_users])

            s = tf.reduce_sum(s, 1)
            s = tf.reshape(s, shape=[self.batch_size, 1])

            score_vector_shape = s.shape[-1]
        
        #about to concatenate and pass through the final layer for score evaluation
        s_vj = tf.concat(axis=1,values=[vj, s])
        
        with tf.variable_scope('FC7layer'):
            Wt = tf.get_variable('Wt', shape=[capture_vector_shape+ score_vector_shape, 2], initializer=tf.random_normal_initializer)
            bt = tf.get_variable('bt', shape = [2], initializer=tf.random_normal_initializer)
        
            self.logits = tf.matmul(s_vj, Wt) + bt

    def loss(self):
        #pass
        #'''
        with tf.name_scope('loss'):
            entropy = tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=tf.one_hot(self.y, 2))
            reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
            #reg_loss = 0.5*reg*tf.nn.l2_loss((tf.trainable_variables()[8]))
            self.loss = tf.reduce_mean(entropy, name='loss') +  0.5*tf.reduce_sum(reg_loss)
        #'''
    def optimizer(self):
        #pass
        with tf.name_scope('optimizer'):
            self.optimizer = tf.train.AdamOptimizer(self.lr).minimize(self.loss, global_step = self.gstep)
    
    def evals(self):
        #pass
        #'''
        with tf.name_scope('predicitons'):
            preds = tf.nn.softmax(self.logits)
            correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(tf.one_hot(self.y, 2), 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))
        #'''
        
    def summary(self):
        #pass
        #'''
        with tf.name_scope('summary'):
            tf.summary.scalar('loss', self.loss)
            tf.summary.scalar('accuracy', self.accuracy)
            tf.summary.histogram('losses', self.loss)
            tf.summary.histogram('Wu', tf.trainable_variables()[8] )
            self.summary_op = tf.summary.merge_all()
        #'''
    def build(self):
        self.get_data()
        self.inference()
        self.loss()
        self.optimizer()
        self.evals()
        self.summary()
    
    def train_once(self, sess, writer, epoch, step):
        #pass
        #'''
        start_time = time.time()
        self.training = True
        total_loss = 0.0
        iterations_perepoch = 792//batch_size
        
        for i in range(iterations_perepoch):

            _, loss_, summary_ = sess.run([self.optimizer, self.loss, self.summary_op],
                                          feed_dict={self.x:X[:,i*batch_size:i*batch_size+batch_size ,:], 
                                                     self.y:label[i*batch_size:i*batch_size+batch_size],
                                                     self.x1: X1, 
                                                     self.mask: mask[:,i*batch_size:i*batch_size+batch_size]})
            #print summary_
            writer.add_summary(summary_, global_step=step)
            total_loss += loss_

        print('Loss at epoch{}: {} and took {} seconds'.format(epoch, total_loss/iterations_perepoch, time.time() - start_time))
        l_initializer)  
    #'''
    def eval_once(self,sess, writer, epoch, step):
        start_time = time.time()
        self.training = True
        total_acc = 0.0
        iterations_perepoch = 199//batch_size
        
        for i in range(iterations_perepoch):

            accuracy_, summary_= sess.run([self.accuracy, self.summary_op],
                                          feed_dict={self.x:X_test[:,i*batch_size:i*batch_size+batch_size ,:], 
                                                     self.y:label_test[i*batch_size:i*batch_size+batch_size],
                                                     self.x1: X1, 
                                                     self.mask: maskTest[:,i*batch_size:i*batch_size+batch_size]})
            print accuracy_
            writer.add_summary(summary_, global_step=step)
            total_acc += accuracy_

        print('Accuracy: {} and took {} seconds'.format(total_acc/iterations_perepoch, time.time() - start_time))
    
    def train(self, n_epochs):
        #pass
        #'''
        writer = tf.summary.FileWriter('./graphs/capture_score', tf.get_default_graph())
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            step = self.gstep.eval()
            
            for epoch in range(n_epochs):
                self.train_once(sess, writer, epoch, step)
                self.eval_once(sess, writer, epoch, step)
        writer.close()
        #'''

In [22]:
if __name__ == '__main__':
    model = Capture_Score()
    model.build()
    model.train(n_epochs)

Loss at epoch0: 26.65126458904411 and took 4.4514319896698 seconds
Accuracy: 0.521052638558965 and took 0.9195117950439453 seconds
Loss at epoch1: 21.890521797952772 and took 4.31838321685791 seconds
Accuracy: 0.5526315875743565 and took 0.8597469329833984 seconds
Loss at epoch2: 18.704306831842736 and took 4.309796094894409 seconds
Accuracy: 0.5421052677066702 and took 0.889373779296875 seconds
Loss at epoch3: 15.840854499913469 and took 4.307809829711914 seconds
Accuracy: 0.5315789541131571 and took 0.8881242275238037 seconds
Loss at epoch4: 13.857473687280583 and took 4.331960439682007 seconds
Accuracy: 0.515789478232986 and took 0.8961484432220459 seconds
Loss at epoch5: 12.251555334163617 and took 4.301618576049805 seconds
Accuracy: 0.5368421156155435 and took 0.8938302993774414 seconds
Loss at epoch6: 11.191631425785113 and took 4.288168430328369 seconds
Accuracy: 0.5368421171840868 and took 0.8787510395050049 seconds
Loss at epoch7: 10.69962283025814 and took 4.316431045532227 s

In [23]:
labels_test.shape

AttributeError: 'dict' object has no attribute 'shape'